In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torchvision
from pytorch_lightning.callbacks import ModelCheckpoint
from src.model.lit_module import LitModule
from src.data.dataset import VideoLabelDataset
import src.constants as const
from torch.utils.data import DataLoader
from src.data.dataset import (VideoLabelDataset,
                              VideoFolderPathToTensor,
                              VideoResize)

In [8]:
!ls lightning_logs/version_8/checkpoints/

'epoch=89-step=719.ckpt'


In [11]:
checkpoint_path = './lightning_logs/version_8/checkpoints/epoch=89-step=719.ckpt'

In [12]:
model = LitModule.load_from_checkpoint(checkpoint_path)

In [13]:
dataset = VideoLabelDataset(
            const.LABELS_TABLE_QA_PATH,
            img_transform=torchvision.transforms.Compose([
                VideoFolderPathToTensor(),
                VideoResize(const.IMG_SIZE)]))

In [14]:
dataloader = DataLoader(dataset, batch_size=10, num_workers=2)

In [15]:
videos, questions, answers, hidden_states  = iter(dataloader).next()

In [16]:
predictions = model.eval()(videos)
predictions

tensor([[0.9497, 0.3239, 0.4910],
        [0.6706, 0.7349, 0.0174],
        [0.2884, 0.5717, 0.9576],
        [0.2325, 0.5860, 0.2299],
        [0.4146, 0.6295, 0.5539],
        [0.2018, 0.5891, 0.8390],
        [0.8191, 0.4503, 0.4888],
        [0.2081, 0.1671, 1.0520],
        [0.5869, 0.7015, 0.5755],
        [0.4927, 0.1388, 0.6038]], grad_fn=<AddmmBackward>)

In [17]:
hidden_states

tensor([[0.9459, 0.0640, 0.4583],
        [0.6545, 0.8860, 0.0000],
        [0.3412, 0.7331, 0.9167],
        [0.2576, 0.5335, 0.2083],
        [0.4305, 0.8481, 0.5417],
        [0.2704, 0.4198, 0.7917],
        [0.8220, 0.6216, 0.5000],
        [0.1734, 0.0584, 0.9583],
        [0.6361, 0.7802, 0.5833],
        [0.5189, 0.0813, 0.6250]], dtype=torch.float64)

In [21]:
mse_loss = torch.nn.MSELoss(reduction='sum')
mse_hidden = mse_loss(predictions[0:2,:].type(torch.float32),
                      hidden_states[0:2,:].type(torch.float32))
mse_hidden

tensor(0.0920, grad_fn=<MseLossBackward>)

In [19]:
((772.6472 - 543.4424)**2 + (912.8349-996.8575)**2 + (36.0000-37.2697)**2 + \
(627.9586-543.4633)**2 + (1077.1292-996.8958)**2 + (25.0000-37.2711)**2)

73323.68388475002

In [20]:
model.eval()(videos)

tensor([[0.9497, 0.3239, 0.4910],
        [0.6706, 0.7349, 0.0174],
        [0.2884, 0.5717, 0.9576],
        [0.2325, 0.5860, 0.2299],
        [0.4146, 0.6295, 0.5539],
        [0.2018, 0.5891, 0.8390],
        [0.8191, 0.4503, 0.4888],
        [0.2081, 0.1671, 1.0520],
        [0.5869, 0.7015, 0.5755],
        [0.4927, 0.1388, 0.6038]], grad_fn=<AddmmBackward>)

In [22]:
predictions[0:2,:]

tensor([[543.4559, 996.8821,  37.2706],
        [543.4633, 996.8958,  37.2711]], grad_fn=<SliceBackward>)